In [5]:
import torch

def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index],
                      db[remove_index+1:]))

def get_parallel_dbs(db):
    parallel_dbs = list()
    
    for i in range(len(db)):
        pdb = get_parallel_db(db, i)
        parallel_dbs.append(pdb)
    
    return parallel_dbs

def create_db_and_parallels(num_entries):
    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    
    return db, pdbs

def query(db, threshold=5):
    return (db.sum() > threshold).float()

def sensitivity(query, n_entries=1000):
    db, pdbs = create_db_and_parallels(n_entries)
    
    full_db_result = query(db)
    
    sensitivity = 0
    for pdb in pdbs:
        pdb_result = query(pdb)

        db_distance = torch.abs(pdb_result - full_db_result)

        if(db_distance > sensitivity):
            sensitivity = db_distance
            
    return sensitivity

In [6]:
db, _ = create_db_and_parallels(100)
db

tensor([1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0,
        1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1,
        1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 0], dtype=torch.uint8)

In [7]:
pdb = get_parallel_db(db, remove_index=10)
db[10]

tensor(1, dtype=torch.uint8)

In [9]:
# differencing attack using sum query

sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [12]:
# differencing attack using mean query

(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(0.0048)

In [18]:
# differencing attack using a threshold

threshold = sum(db).float() - 1

(sum(db).float() > threshold) - (sum(pdb).float() > threshold)

tensor(1, dtype=torch.uint8)

tensor(52, dtype=torch.uint8)